# Setting params dictionary & Logging Setting

In [1]:
import logging
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.DEBUG) # similar to fstool connect_module debug 5 10h

In [17]:
params = {}
params["connect_cylance_url"] = 'http://10.0.1.3:3000'
params["connect_authorization_token"] = '123456'
params["mac"] = '001122334455'

In [22]:
# Setting up the ssl_context
import ssl
ssl_context = ssl.SSLContext(ssl.PROTOCOL_SSLv23)

# 1. Sample Test Script

In [11]:
import jwt # PyJWT version 1.6.1 as of the time of authoring
import uuid
import json
import urllib.request
import time
from time import gmtime, strftime, sleep
from datetime import datetime, timedelta

# CONFIGURATION
# All server configuration fields will be available in the 'params'dictionary.

jwt_token = params["connect_authorization_token"] # auth token

response = {}

# Like the action response, the response object must have a "succeeded" field to denote success. 
# It can also optionally have a "result_msg" field to display a custom test result message.

if "connect_authorization_token" in params and params["connect_authorization_token"] != "":
    response["succeeded"] = True
    response["result_msg"] = "Successfully connected."
else:
    response["succeeded"] = False
    response["result_msg"] = "Could not connect to Cylance server."

In [12]:
# Checking the response
response

{'succeeded': True, 'result_msg': 'Successfully connected.'}

## 2. Sample Polling Script

In [13]:
import jwt # PyJWT version 1.6.1 as of the time of authoring
import uuid
import time
from time import gmtime, strftime, sleep
from datetime import datetime, timedelta

# Mapping between Cylance API response fields to CounterACT properties
cylance_to_ct_props_map = {
    "state": "connect_cylance_state",
    "mac_addresses": "connect_cylance_mac_addresses",
    "id": "connect_cylance_id"
 }

# CONFIGURATION
url = params["connect_cylance_url"] # Server URL

response = {}

endpoints=[]

# Check if we have valid auth token or not before processing.

if "connect_authorization_token" in params and params["connect_authorization_token"] != "":
    # ***** PART 2 - QUERY FOR DEVICES ***** #
    jwt_token = params["connect_authorization_token"]
    GETMAC_URL = url + "/devices/v2/"
    device_headers = {"Content-Type": "application/json; charset=utf-8",\
                      "Authorization": "Bearer " + str(jwt_token)}
    # Get MAC data
    request = urllib.request.Request(GETMAC_URL, headers=device_headers)
    try:
        r = urllib.request.urlopen(request, context=ssl_context)
        request_response = json.loads(r.read())
        
        # For polling, the response dictionary must contain a list called "endpoints", which will 
        # contain new endpoint information. Each endpoint must have a field named either "mac" or "ip".
        # The endpoint object/dictionary may also have a "properties" field, which contains
        # property information in the format {"propert_name": "property_value"}. 
        # The full response object, for example would be:
        # {"endpoints":
        #   [
        #    {"mac": "001122334455",
        #     "properties":
        #        {"property1": "property_value", "property2":"property_value2"}
        #     }
        #   ]
        # }
        
        for endpoint_data in request_response["page_items"]:
            endpoint = {}
            mac_with_dash = endpoint_data["mac_addresses"][0]
            mac = "".join(mac_with_dash.split("-"))
            endpoint["mac"] = mac
            properties = {}
            
            for key, value in endpoint_data.items():
                if key in cylance_to_ct_props_map and key is not "mac_addresses":
                    properties[cylance_to_ct_props_map[key]] = value
                    endpoint["properties"] = properties
                    endpoints.append(endpoint)
            
            response["endpoints"] = endpoints
    except:
        response["error"] = "Could not retrieve endpoints."
else:
    response["error"] = "Unauthorized"

In [14]:
# checking the response - need correct connect_authorization_token
response

{'error': 'Could not retrieve endpoints.'}

## 3. Sample Resolve Script

In [23]:
import jwt # PyJWT version 1.6.1 as of the time of authoring
import uuid
import time
from time import gmtime, strftime, sleep
from datetime import datetime, timedelta

# Mapping between Cylance API response fields to CounterACT properties
cylance_to_ct_props_map = {
    "state": "connect_cylance_state",
    "last_logged_in_user": "connect_cylance_last_logged_in_user",
    "mac_addresses": "connect_cylance_mac_addresses",
    "is_safe": "connect_cylance_is_safe",
    "id": "connect_cylance_id"
}

# CONFIGURATION
# All server configuration fields will be available in the 'params' dictionary.
url = params["connect_cylance_url"] # Server URL

response = {}

# Check if we have valid auth token or not before processing.
if "connect_authorization_token" in params and params["connect_authorization_token"] != "":
    # For properties and actions defined in the 'property.conf' file,
    # CounterACT properties can be added as dependencies. These values will be
    # found in the params dictionary if CounterACT was able to resolve
    # the properties. If not, they will not be found in the params dictionary.
    
    jwt_token = params["connect_authorization_token"]
    if "mac" in params:
        mac = '-'.join(params["mac"][i:i+2] for i in range(0,12,2))
        GETMAC_URL = url + "/devices/v2/macaddress/" + mac
        device_headers = {"Content-Type": "application/json;charset=utf-8", \
                          "Authorization": "Bearer " + str(jwt_token)}
        # Get MAC data
        request = urllib.request.Request(GETMAC_URL, headers=device_headers)
        try:
            r = urllib.request.urlopen(request, context=ssl_context)
            request_response = json.loads(r.read())
            
            # All responses from scripts must contain the JSON object 'response'. 
            # Host property resolve scripts will need to populate a 'properties' JSON object 
            # within the JSON object 'response'. The 'properties' object will be a key, value mapping 
            # between the CounterACT property name and the value of the property.
            
            properties = {}
            if request_response:
                return_values = request_response[0]
                for key, value in return_values.items():
                    if key in cylance_to_ct_props_map:
                        properties[cylance_to_ct_props_map[key]] = value
                
            response["properties"] = properties
        
        except Exception as e:
            response["error"] = "Could not resolve properties:{}".format(str(e))
            
    else: 
        response["error"] = "No mac address to query the endpoint for."
else: 
    response["error"] = "Unauthorized"

In [24]:
response

{'error': 'Could not resolve properties:HTTP Error 404: Not Found'}

## 4. Sample App Instance Cache Script

In [ ]:
import jwt # PyJWT version 1.6.1 as of the time of authoring
import uuid
import time
from time import gmtime, strftime, sleep
from datetime import datetime, timedelta
from urllib.request import HTTPError, URLError

CONFIGURATION
url = params["connect_cylance_url"] # Server URL
response = {}
# Check if we have valid auth token or not before processing.
if "connect_authorization_token" in params and params["connect_authorization_token"] != "":
    
    # ***** PART 2 - QUERY FOR USERS ***** #
    jwt_token = params["connect_authorization_token"]
    GETUSERS_URL = url + "/users/v2/"
    device_headers = {"Content-Type": "application/json; charset=utf-8", \
                  "Authorization": "Bearer " + str(jwt_token)}
    
    # Get users to save as app instance cache
    request = urllib.request.Request(GETUSERS_URL, headers=device_headers)
    
    try:
        r = urllib.request.urlopen(request, context=ssl_context)
        request_response = json.loads(r.read())
        response_obj = {}
        for user_data in request_response["page_items"]:
            user = {}
            email = user_data["email"]
            user["id"] = user_data["id"];
            user["first_name"] = user_data["first_name"]
            user["last_name"] = user_data["last_name"]
            response_obj[email] = user
        
        # For app instance cache, use the 'connect_app_instance_cache' to be the response key.
        # The value needs to be a string. It can be a json string containing different fields or any other format,
        # depending on how you want to use the data in other scripts.
        
        response["connect_app_instance_cache"] = json.dumps(response_obj)
        logging.debug("response: {}".format(response))
    
    except HTTPError as e:
        response["error"] = "Could not connect to Cylance. Response code:{}".format(e.code)
    
    except URLError as e:
        response["error"] = "Could not connect to Cylance.{}".format(e.reason)
    
    except Exception as e:
        response["error"] = "Could not connect to Cylance.{}".format(str(e))

else:
    # In the response, put 'error' to indicate the error message.
    # 'connect_app_instance_cache' is optinal when it has error.
    # if connect_app_instance_cache is in the response object, it will overwrite previous cache value.
    # Otherwise, the previous cache value will remain the same.
    response["connect_app_instance_cache"] = "{}"
    response["error"] = "No authorization token found"

## 5. Sample Action Script to Add a User

In [ ]:
import jwt # PyJWT version 1.6.1 as of the time of authoring
import uuid
import time
from time import gmtime, strftime, sleep
from datetime import datetime, timedelta
from urllib.request import HTTPError, URLError

# CONFIGURATION
# All server configuration fields will be available in the 'params' dictionary.
url = params["connect_cylance_url"] # Server URL
response = {}
properties = {}
action_status = {}

# Check if we have valid auth token or not before processing.
if "connect_authorization_token" in params and params["connect_authorization_token"] != "":
    
    # ***** Execute add user action ***** #
    jwt_token = params["connect_authorization_token"]
    
    # connect_app_instance_cache data is available when you enable app_instance_cache feature in system.conf
    # the data is available in the 'params' dictionary.
    # In this example, we are getting the existing users.
    user_list = params.get("connect_app_instance_cache")
    user_email = params["cylance_email"]
    
    if user_list is not None and user_email in user_list:
        
        # return action failed if user already exists
        logging.debug("User {} already exists. ".format(user_email))
        
        response["succeeded"] = False
        response["troubleshooting"] = "User already exists."
        
        action_status["status"] = "Failed. User already exists."
        action_status["time"] = int(time.time())
        
        # Add properties dictionary to response to resolve properties. It is optional
        properties["connect_cylance_add_user_action"] = action_status
        response["properties"] = properties
    else:
        ADD_USER_URL = url + "/users/v2/"
        device_headers = {"Content-Type": "application/json;charset=utf-8", \
                          "Authorization": "Bearer " + str(jwt_token)}
        body = dict()
        # For actions, you can specify user inputted parameters that must be defined in the 'property.conf' file. 
        # These parameters will take in user input from the CounterACT console 
        # and will be available in the 'params' dictionary.
        
        body["email"] = params["cylance_email"]
        body["user_role"] = "00000000-0000-0000-0000-000000000001"
        body["first_name"] = params["cylance_first_name"]
        body["last_name"] = params["cylance_last_name"]
        
        zones = dict()
        zone_array = list()
        
        zones["id"] = "0927bf62-83f4-4766-a825-0b5d2e9749d0"
        zones["role_type"] = "00000000-0000-0000-0000-000000000002"
        zones["role_name"] = "User"
        
        zone_array.append(zones)
        body["zones"] = zone_array
        json_body = json.dumps(body).encode('utf-8')
        request = urllib.request.Request(ADD_USER_URL, headers=device_headers, data=json_body)
        
        try:
            r = urllib.request.urlopen(request, context=ssl_context)
            # For actions, the response object must have a field named "succeeded" to denote if the action suceeded or not.
            # The field "troubleshooting" is optional to display user defined messages in CounterACT for actions. The field
            # "cookie" is available for continuous/cancellable actions to store information for the same action. For this example,
            # the cookie stores the id of the user, which will be used to delete the same user when this action is cancelled.
            response["succeeded"] = True
            request_response = json.loads(r.read())
            _id = request_response['id']
            logging.debug("The cookie content is {}".format(_id))
            response["cookie"] = _id
            action_status["status"] = "Succeeded"
            action_status["time"] = int(time.time())    
            properties["connect_cylance_add_user_action"] = action_status
            properties["connect_cylance_last_logged_in_user"] = params["cylance_email"]
            response["properties"] = properties
            
        except HTTPError as e:
            response["succeeded"] = False
            response["troubleshooting"] = "Failed action. Response code: {}".format(e.code)
            action_status["status"] = "Failed. HTTPError."
            action_status["time"] = int(time.time())
            properties["connect_cylance_add_user_action"] = action_status
            response["properties"] = properties
        
        except URLError as e:
            response["troubleshooting"] = "Failed action.{}".format(e.reason)
            response["succeeded"] = False
            action_status["status"] = "Failed. URLError."
            action_status["time"] = int(time.time())
            properties["connect_cylance_add_user_action"] =action_status
            response["properties"] = properties
            
        except Exception as e:
            response["troubleshooting"] = "Failed action.{}".format(str(e))
            response["succeeded"] = False
            action_status["status"] = "Failed. Exception."
            action_status["time"] = int(time.time())
            properties["connect_cylance_add_user_action"] = action_status
            response["properties"] = properties
else:
    response["succeeded"] = False
    response["troubleshooting"] = "Unauthorized"
    action_status["status"] = "Failed. Unauthorized."
    action_status["time"] = int(time.time())
    properties["connect_cylance_add_user_action"] = action_status
    response["properties"] = properties

## 6. Sample Action Script to Delete a User

In [ ]:
import jwt # PyJWT version 1.6.1 as of the time of authoring
import uuid
import time
from time import gmtime, strftime, sleep
from datetime import datetime, timedelta
from urllib.request import HTTPError, URLError

# CONFIGURATION
# All server configuration fields will be available in the 'params' dictionary.
url = params["connect_cylance_url"] # Server URL

response = {}
properties = {}
action_status = {}

# Check if we have valid auth token or not before processing.
if "connect_authorization_token" in params and params["connect_authorization_token"] != "":
    # ***** PART 2 - DELETE USER ***** #
    # Here, the cookie that was set in adding the user is being used. The user id is used to delete the user.
    jwt_token = params["connect_authorization_token"]
    
    DELETE_USER_URL = url + "/users/v2/" + params["cookie"]
    
    device_headers = {"Authorization": "Bearer " + str(jwt_token)}
    
    request = urllib.request.Request(DELETE_USER_URL, headers=device_headers, method='DELETE')
    
    try:
        r = urllib.request.urlopen(request, context=ssl_context)
        response["succeeded"] = True
        action_status["status"] = "Succeeded"
        action_status["time"] = int(time.time())
        properties["connect_cylance_add_user_action"] = action_status
        response["properties"] = properties
        
    except HTTPError as e:
        response["troubleshooting"] = "Failed action. Response code:{}".format(e.code)
        response["succeeded"] = False
        action_status["status"] = "Failed. HTTPError."
        action_status["time"] = int(time.time())
        properties["connect_cylance_add_user_action"] = action_status
        response["properties"] = properties
    
    except URLError as e:
        response["troubleshooting"] = "Failed action.{}".format(e.reason)
        response["succeeded"] = False
        action_status["status"] = "Failed. URLError."
        action_status["time"] = int(time.time())
        properties["connect_cylance_add_user_action"] = action_status
        response["properties"] = properties
    
    except Exception as e:
        response["troubleshooting"] = "Failed action.{}".format(str(e))
        response["succeeded"] = False
        action_status["status"] = "Failed. Exception."
        action_status["time"] = int(time.time())
        properties["connect_cylance_add_user_action"] = action_status
        response["properties"] = properties
else:
    response["succeeded"] = False
    response["troubleshooting"] = "Unauthorized"
    action_status["status"] = "Failed. Unauthorized."
    action_status["time"] = int(time.time())
    properties["connect_cylance_add_user_action"] = action_status
    response["properties"] = properties

## 7. Sample Authorization Script

In [ ]:
import jwt # PyJWT version 1.6.1 as of the time of authoring
import uuid
import json
import urllib.request
import time
from time import gmtime, strftime, sleep
from datetime import datetime, timedelta

# CONFIGURATION
# All server configuration fields will be available in the 'params' dictionary.
url = params["connect_cylance_url"] # Server URL
tenant = params["connect_cylance_tenant_id"] # Tenant ID
app = params["connect_cylance_application_id"] # Application ID
secret = params["connect_cylance_application_secret"] # Application Secret

# ***** START - AUTH API CONFIGURATION ***** #
timeout = 1800 # 30 minutes from now
now = datetime.utcnow()
timeout_datetime = now + timedelta(seconds=timeout)
epoch_time = int((now - datetime(1970, 1, 1)).total_seconds())
epoch_timeout = int((timeout_datetime - datetime(1970, 1, 1)).total_seconds())
jti_val = str(uuid.uuid4())
claims = {
    "exp": epoch_timeout,
    "iat": epoch_time,
    "iss": "http://cylance.com",
    "sub": app,
    "tid": tenant,
    "jti": jti_val,
}

encoded = jwt.encode(claims, secret, algorithm='HS256')
payload = {"auth_token": encoded.decode("utf-8")}
headers = {"Content-Type": "application/json; charset=utf-8"}

# Making an API call to get the JWT token
request = urllib.request.Request(url + "/auth/v2/token", headers=headers, \
                                 data=bytes(json.dumps(payload), encoding="utf-8"))
response = {}

try:
    # To use the server validation feature, use the keyword 'ssl_context' in the http reqeust
    resp = urllib.request.urlopen(request, context=ssl_context)
    jwt_token = json.loads(resp.read())['access_token'] # access_token to be passed to GET request
    response["token"] = jwt_token
except:
    response["token"] = ""
    